In [170]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, 
                           accuracy_score)

In [154]:
df = pd.read_csv('data/weather_data_hcm_[2017-2022]_preprocess.csv')

In [155]:
df.head()

,Time,Day,Month,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather
0,00:00,1,1,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,46.6,Partly cloudy
1,03:00,1,1,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,26.9,Partly cloudy
2,06:00,1,1,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,29.2,Partly cloudy
3,09:00,1,1,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,66.7,Partly cloudy
4,12:00,1,1,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,73.0,Partly cloudy


In [156]:
label = df['Weather']
features = df

In [157]:
#remove data
features = features[:-1]
label = label[1:]
features.drop(['Day', 'Year'], axis=1)

,Time,Month,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather
0,00:00,1,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,46.6,Partly cloudy
1,03:00,1,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,26.9,Partly cloudy
2,06:00,1,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,29.2,Partly cloudy
3,09:00,1,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,66.7,Partly cloudy
4,12:00,1,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,73.0,Partly cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...
14255,06:00,12,94.0,23.0,25.0,0.0,6.0,1014.0,8.0,12.0,37.5,Clear
14256,09:00,12,70.0,23.0,32.0,0.0,4.0,1016.0,9.0,11.0,33.7,Clear
14257,12:00,12,49.0,20.0,34.0,0.0,14.0,1013.0,7.0,8.0,59.0,Clear
14258,15:00,12,36.0,17.0,33.0,0.0,11.0,1011.0,9.0,11.0,104.4,Clear


In [158]:
RAIN_LABEL = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain', 
             'Patchy light rain with thunder','Patchy rain possible','Heavy rain','Heavy rain at times',
             'Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast','Torrential rain shower']
RAINY_MONTH = [5,6,7,8,9,10,11]
MORNING_HOUR = [6,9,12,15]

In [159]:
for i, value in enumerate(label):
    if value in RAIN_LABEL:
        label[i] = 'yes'
    else:
        label[i] = 'no'

In [160]:
for i, value in enumerate(features.Time):
    hour = int(value.split(':')[0])
    if hour in MORNING_HOUR:
        features.Time[i] = 'morning'
    else:
        features.Time[i] = 'evening'

/tmp/ipykernel_2432690/3364625140.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.Time[i] = 'evening'
/data4/thang/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2432690/3364625140.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.Time[i] = 'morning'


In [161]:
for i, value in enumerate(features.Weather):
    if value in RAIN_LABEL:
        features.Weather[i] = 'rainy'
    else:
        features.Weather[i] = 'sunny'

/tmp/ipykernel_2432690/348643353.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.Weather[i] = 'sunny'
/tmp/ipykernel_2432690/348643353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.Weather[i] = 'rainy'


In [162]:
for i, value in enumerate(features.Month):
    if int(value) in RAINY_MONTH:
        features.Month[i] = 'rainy_month'
    else:
        features.Month[i] = 'sunny_month'

/tmp/ipykernel_2432690/2840697016.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.Month[i] = 'sunny_month'
/data4/thang/envs/py38/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/data4/thang/envs/py38/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_wit

In [163]:
features.head()

,Time,Day,Month,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather
0,evening,1,sunny_month,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,46.6,sunny
1,evening,1,sunny_month,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,26.9,sunny
2,morning,1,sunny_month,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,29.2,sunny
3,morning,1,sunny_month,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,66.7,sunny
4,morning,1,sunny_month,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,73.0,sunny


In [164]:
data = features
data['Label'] = label
data = pd.get_dummies(data, drop_first = True)

/tmp/ipykernel_2432690/2393237508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Label'] = label


In [167]:
train_set, test_set = train_test_split(data, test_size=0.2)

In [168]:
Xtrain = train_set[['Day', 'Year', 'Humidity', 'Dew point', 'Temperature', 'Rain', 'Cloud',
       'Pressure', 'Wind', 'Gust', 'Dir', 'Time_morning', 'Month_sunny_month',
       'Weather_sunny']]
ytrain = train_set[['Label_yes']]
   
# building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: inf
         Iterations 7


/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Label_yes   No. Observations:                11408
Model:                          Logit   Df Residuals:                    11394
Method:                           MLE   Df Model:                           13
Date:                Sat, 23 Jul 2022   Pseudo R-squ.:                     inf
Time:                        09:59:26   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Day                   0.0025      0.003      0.846      0.398      -0.003       0.008
Year                 -0.0011      0.007     -0.161      0.872      -0.014       0.012
Humidity             -0.0021      0.004     -0.576      0.565      -0.009       0.005
Dew point             0.1523      0.023      6.688      0.000       0.108       0.197
Temperature           0.0773      0.015      5.094      0.000       0.048       0.107
Rain                  0.3923      0.028     13.836      0.000       0.337       0.448
Cloud                 0.0369      0.001     28.681      0.000       0.034       0.039
Pressure             -0.0059      0.013     -0.441      0.659      -0.032       0.020
Wind                 -0.0887      0.025     -3.559      0.000      -0.138      -0.040
Gust                  0.0796      0.018      4.468      0.000       0.045       0.115
Dir                   0.0010      0.000      2.842      0.004       0.000       0.002
Time_morning          0.3535      0.070      5.074      0.000       0.217       0.490
Month_sunny_month    -0.6197      0.071     -8.747      0.000      -0.759      -0.481
Weather_sunny        -0.8154      0.065    -12.468      0.000      -0.944      -0.687
=====================================================================================
"""

In [171]:
Xtest = test_set[['Day', 'Year', 'Humidity', 'Dew point', 'Temperature', 'Rain', 'Cloud',
       'Pressure', 'Wind', 'Gust', 'Dir', 'Time_morning', 'Month_sunny_month',
       'Weather_sunny']]
ytest = test_set[['Label_yes']]

yhat = log_reg.predict(Xtest)
prediction = list(map(round, yhat))
cm = confusion_matrix(ytest, prediction) 
print ("Confusion Matrix : \n", cm) 

# accuracy score of the model
print('Test accuracy = ', accuracy_score(ytest, prediction))

Confusion Matrix : 
 [[1630  220]
 [ 300  702]]
Test accuracy =  0.8176718092566619
